In [15]:
# !pip install -r ./requirements.txt

In [2]:
# !pip uninstall virny -y

In [4]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@feature/prepare_experiments_for_inprocessors

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@feature/prepare_experiments_for_inprocessors

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../../../..")

print('Current location: ', os.getcwd())

Current location:  /home/dh3553/projects/fairness-variance


## Import dependencies

In [4]:
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import ACSIncomeDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS

from source.experiment_interface import run_exp_iter_with_inprocessor

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'


## Define Input Variables

In [5]:
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'ADB_acs_income'
DB_COLLECTION_NAME = 'one_repair_lvl_many_models'
FAIRNESS_INTERVENTION_NAME = 'ADB'
FAIR_INTERVENTION_PARAMS_LST = ['debiased_classifier']
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                     FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME)

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', 'diff_fairness_interventions_exp',
                                FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, 'folk_GA_2018_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [6]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [7]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [8]:
import uuid

custom_table_fields_dct = {
#     'session_uuid': str(uuid.uuid4()),
    'session_uuid': 'bbba3cc4-760b-4e93-bb97-3a2077202cce',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  bbba3cc4-760b-4e93-bb97-3a2077202cce


## Initialize custom objects

In [9]:
data_loader = ACSIncomeDataset(state=['GA'], year=2018, with_nulls=False,
                               subsample_size=15_000, subsample_seed=42)
data_loader.X_data.head()

,SCHL,COW,MAR,OCCP,POBP,RELP,SEX,RAC1P,AGEP,WKHP
0,23,7,3,230,36,0,1,1,55,55.0
1,16,1,5,4110,13,2,2,1,20,35.0
2,16,4,3,4130,51,0,2,1,59,30.0
3,18,4,1,4020,13,0,1,2,43,40.0
4,14,1,1,8300,20,1,2,2,33,20.0


In [10]:
data_loader.X_data.shape

(15000, 10)

## Run experiment iterations

### Experiment iteration 1

In [13]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [ ]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='ACSIncomeDataset',
                              inprocessor_name='AdversarialDebiasing',
                              verbose=True)

2024-01-08 22:04:06 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': "['debiased_classifier']",
 'model_init_seed': 100,
 'session_uuid': 'bbba3cc4-760b-4e93-bb97-3a2077202cce'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-08 22:04:06 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


intervention_option:  debiased_classifier
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__SCHL_1', 'cat__SCHL_10', 'cat__SCHL_11', 'cat__SCHL_12',
       'cat__SCHL_13', 'cat__SCHL_14', 'cat__SCHL_15', 'cat__SCHL_16',
       'cat__SCHL_17', 'cat__SCHL_18',
       ...
       'cat__RELP_3', 'cat__RELP_4', 'cat__RELP_5', 'cat__RELP_6',
       'cat__RELP_7', 'cat__RELP_8', 'cat__RELP_9', 'num__AGEP', 'num__WKHP',
       'SEX&RAC1P_binary'],
      dtype='object', length=724)
Top indexes of an X_test in the current base flow dataset:  Int64Index([10155, 11689, 12599, 12193,  8678,  8217,  4670, 12087,  5235,
             4189,  7278, 10642,  5284,  7002, 14642, 10594,  7701,  8686,
             8665,  6253],
           dtype='int64')
Top indexes of an y_test in the current base flow dataset:  Int64Index([10155, 11689, 12599, 12193,  8678,  8217,  4670, 12087,  5235,
             4189,  7278, 10642,  5284,  7002, 14642, 10594,  7701,  8686,
             8665,  6253],
           dtype

Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 0.703511; batch adversarial loss: 0.868435
epoch 1; iter: 0; batch classifier loss: 0.433493; batch adversarial loss: 0.870017
epoch 2; iter: 0; batch classifier loss: 0.396227; batch adversarial loss: 0.863279
epoch 3; iter: 0; batch classifier loss: 0.393296; batch adversarial loss: 0.784595
epoch 4; iter: 0; batch classifier loss: 0.335170; batch adversarial loss: 0.729312
epoch 5; iter: 0; batch classifier loss: 0.302273; batch adversarial loss: 0.718420
epoch 6; iter: 0; batch classifier loss: 0.340553; batch adversarial loss: 0.671042
epoch 7; iter: 0; batch classifier loss: 0.317702; batch adversarial loss: 0.662751
epoch 8; iter: 0; batch classifier loss: 0.252170; batch adversarial loss: 0.648393
epoch 9; iter: 0; batch classifier loss: 0.292380; batch adversarial loss: 0.613334
epoch 10; iter: 0; batch classifier loss: 0.278500; batch adversarial loss: 0.582467
epoch 11; iter: 0; batch classifier loss: 0.315808; batch adversarial loss:

### Experiment iteration 2

In [4]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

NameError: name 'EXPERIMENT_SEEDS' is not defined

In [21]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='ACSIncomeDataset',
                              inprocessor_name='AdversarialDebiasing',
                              verbose=True)

2024-01-04 17:15:39 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': '[True]',
 'intervention_param': 'True',
 'model_init_seed': 200,
 'session_uuid': '0626d80f-e288-4f16-b8ab-260eb34d62d3'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-04 17:15:39 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-04 17:15:39 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_option:  True
Using ROC postprocessor
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
     

...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 3

In [11]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [12]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='ACSIncomeDataset',
                              inprocessor_name='AdversarialDebiasing',
                              verbose=True)

2024-01-04 19:29:01 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 300,
 'experiment_iteration': 'Exp_iter_3',
 'fair_intervention_params_lst': '[True]',
 'model_init_seed': 300,
 'session_uuid': '0626d80f-e288-4f16-b8ab-260eb34d62d3'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-04 19:29:01 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-04 19:29:01 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_option:  True
Using ROC postprocessor
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
     

...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 4

In [13]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [14]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='ACSIncomeDataset',
                              inprocessor_name='AdversarialDebiasing',
                              verbose=True)

2024-01-04 20:15:17 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 400,
 'experiment_iteration': 'Exp_iter_4',
 'fair_intervention_params_lst': '[True]',
 'intervention_param': 'True',
 'model_init_seed': 400,
 'session_uuid': '0626d80f-e288-4f16-b8ab-260eb34d62d3'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-04 20:15:17 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-04 20:15:17 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_option:  True
Using ROC postprocessor
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
     

...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 5

In [11]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [12]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='ACSIncomeDataset',
                              inprocessor_name='AdversarialDebiasing',
                              verbose=True)

2024-01-08 22:06:03 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 500,
 'experiment_iteration': 'Exp_iter_5',
 'fair_intervention_params_lst': "['debiased_classifier']",
 'model_init_seed': 500,
 'session_uuid': 'bbba3cc4-760b-4e93-bb97-3a2077202cce'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-08 22:06:03 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


intervention_option:  debiased_classifier
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__SCHL_1', 'cat__SCHL_10', 'cat__SCHL_11', 'cat__SCHL_12',
       'cat__SCHL_13', 'cat__SCHL_14', 'cat__SCHL_15', 'cat__SCHL_16',
       'cat__SCHL_17', 'cat__SCHL_18',
       ...
       'cat__RELP_3', 'cat__RELP_4', 'cat__RELP_5', 'cat__RELP_6',
       'cat__RELP_7', 'cat__RELP_8', 'cat__RELP_9', 'num__AGEP', 'num__WKHP',
       'SEX&RAC1P_binary'],
      dtype='object', length=730)
Top indexes of an X_test in the current base flow dataset:  Int64Index([ 8104,   580,  8954,  6500,  5436,  3490,  1621, 11610,  7067,
            10637,   767, 10997,  6202,  4468,  8440, 10810, 11409,  3682,
            12856, 14104],
           dtype='int64')
Top indexes of an y_test in the current base flow dataset:  Int64Index([ 8104,   580,  8954,  6500,  5436,  3490,  1621, 11610,  7067,
            10637,   767, 10997,  6202,  4468,  8440, 10810, 11409,  3682,
            12856, 14104],
           dtype

...

epoch 8; iter: 0; batch classifier loss: 0.380930; batch adversarial loss: 0.573503
epoch 9; iter: 0; batch classifier loss: 0.362218; batch adversarial loss: 0.532982
epoch 10; iter: 0; batch classifier loss: 0.338543; batch adversarial loss: 0.556759
epoch 11; iter: 0; batch classifier loss: 0.311984; batch adversarial loss: 0.537674
epoch 12; iter: 0; batch classifier loss: 0.251935; batch adversarial loss: 0.549838
epoch 13; iter: 0; batch classifier loss: 0.360565; batch adversarial loss: 0.509099
epoch 14; iter: 0; batch classifier loss: 0.311244; batch adversarial loss: 0.460939
epoch 15; iter: 0; batch classifier loss: 0.375257; batch adversarial loss: 0.472408
epoch 16; iter: 0; batch classifier loss: 0.283332; batch adversarial loss: 0.489746
epoch 17; iter: 0; batch classifier loss: 0.382566; batch adversarial loss: 0.498435
epoch 18; iter: 0; batch classifier loss: 0.329106; batch adversarial loss: 0.496083
epoch 19; iter: 0; batch classifier loss: 0.302756; batch adversari

epoch 106; iter: 0; batch classifier loss: 0.060007; batch adversarial loss: 0.548035
epoch 107; iter: 0; batch classifier loss: 0.046166; batch adversarial loss: 0.453543
epoch 108; iter: 0; batch classifier loss: 0.041056; batch adversarial loss: 0.470654
epoch 109; iter: 0; batch classifier loss: 0.033853; batch adversarial loss: 0.474501
epoch 110; iter: 0; batch classifier loss: 0.046023; batch adversarial loss: 0.391031
epoch 111; iter: 0; batch classifier loss: 0.061408; batch adversarial loss: 0.451438
epoch 112; iter: 0; batch classifier loss: 0.042811; batch adversarial loss: 0.494443
epoch 113; iter: 0; batch classifier loss: 0.027738; batch adversarial loss: 0.523455
epoch 114; iter: 0; batch classifier loss: 0.019006; batch adversarial loss: 0.393450
epoch 115; iter: 0; batch classifier loss: 0.024393; batch adversarial loss: 0.516544
epoch 116; iter: 0; batch classifier loss: 0.037491; batch adversarial loss: 0.563237
epoch 117; iter: 0; batch classifier loss: 0.040432; b

epoch 2; iter: 0; batch classifier loss: 0.390735; batch adversarial loss: 0.602832
epoch 3; iter: 0; batch classifier loss: 0.401106; batch adversarial loss: 0.627377
epoch 4; iter: 0; batch classifier loss: 0.377917; batch adversarial loss: 0.570172
epoch 5; iter: 0; batch classifier loss: 0.295706; batch adversarial loss: 0.573633
epoch 6; iter: 0; batch classifier loss: 0.445659; batch adversarial loss: 0.590615
epoch 7; iter: 0; batch classifier loss: 0.383906; batch adversarial loss: 0.597184
epoch 8; iter: 0; batch classifier loss: 0.338368; batch adversarial loss: 0.526856
epoch 9; iter: 0; batch classifier loss: 0.297256; batch adversarial loss: 0.579419
epoch 10; iter: 0; batch classifier loss: 0.438275; batch adversarial loss: 0.519245
epoch 11; iter: 0; batch classifier loss: 0.450512; batch adversarial loss: 0.624756
epoch 12; iter: 0; batch classifier loss: 0.619983; batch adversarial loss: 0.555809
epoch 13; iter: 0; batch classifier loss: 0.510903; batch adversarial los

epoch 100; iter: 0; batch classifier loss: 0.108578; batch adversarial loss: 0.482339
epoch 101; iter: 0; batch classifier loss: 0.148024; batch adversarial loss: 0.520084
epoch 102; iter: 0; batch classifier loss: 0.194698; batch adversarial loss: 0.482856
epoch 103; iter: 0; batch classifier loss: 0.153395; batch adversarial loss: 0.448347
epoch 104; iter: 0; batch classifier loss: 0.181356; batch adversarial loss: 0.494742
epoch 105; iter: 0; batch classifier loss: 0.226948; batch adversarial loss: 0.374329
epoch 106; iter: 0; batch classifier loss: 0.213855; batch adversarial loss: 0.458820
epoch 107; iter: 0; batch classifier loss: 0.149472; batch adversarial loss: 0.399089
epoch 108; iter: 0; batch classifier loss: 0.131390; batch adversarial loss: 0.410865
epoch 109; iter: 0; batch classifier loss: 0.127375; batch adversarial loss: 0.447016
epoch 110; iter: 0; batch classifier loss: 0.269096; batch adversarial loss: 0.459139
epoch 111; iter: 0; batch classifier loss: 0.157890; b

epoch 196; iter: 0; batch classifier loss: 0.009850; batch adversarial loss: 0.422947
epoch 197; iter: 0; batch classifier loss: 0.009087; batch adversarial loss: 0.476352
epoch 198; iter: 0; batch classifier loss: 0.038829; batch adversarial loss: 0.473622
epoch 199; iter: 0; batch classifier loss: 0.036507; batch adversarial loss: 0.537723


### Experiment iteration 6

In [13]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [14]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='ACSIncomeDataset',
                              inprocessor_name='AdversarialDebiasing',
                              verbose=True)

2024-01-05 04:49:43 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 600,
 'experiment_iteration': 'Exp_iter_6',
 'fair_intervention_params_lst': '[True]',
 'intervention_param': 'True',
 'model_init_seed': 600,
 'session_uuid': '0626d80f-e288-4f16-b8ab-260eb34d62d3'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-05 04:49:43 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-05 04:49:43 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_option:  True
Using ROC postprocessor
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
     

...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]